# Metropolitan Data, Assignment 3
The city of Amsterdam expects a lot of visitors for the open water swimming event. To have an overview of the accommodation possibilities for the extra tourists entering the city, Amsterdam wants to get an insight into the number of tourists that will make use of AirBnB. 

We start by importing Pandas since we need to read some files, we import plotly and os.path as well.

In [4]:
import pandas as pd
import os.path
import osmnx as ox
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import networkx as nx

We load the Airbnb data from Amsterdam and convert the csv data file to a xlsx file.

In [14]:
# Load the Airbnb data from a CSV file
df_air = pd.read_csv('listings.csv', header=0)
print(df_air.head())
# Save the DataFrame to an Excel file
path = os.getcwd()
file = os.path.join(path, 'air_bnb_prices.xlsx')
df_air.to_excel(file, index=False)

       id                                               name  host_id  \
0  761411  Condo in Amsterdam · ★4.74 · 1 bedroom · 1 bed...  4013546   
1  768274  Rental unit in Amsterdam · ★4.83 · 1 bedroom ·...  3678216   
2  768737  Boat in Amsterdam · ★4.82 · 1 bedroom · 1 bed ...  3877342   
3  771217  Houseboat in Amsterdam · ★5.0 · 3 bedrooms · 3...  4068486   
4  771343  Rental unit in Amsterdam · ★4.89 · 1 bedroom ·...  2313061   

  host_name  neighbourhood_group neighbourhood  latitude  longitude  \
0    Xsjong                  NaN    Noord-Oost  52.40164    4.95106   
1     J & R                  NaN    Westerpark  52.38855    4.88521   
2    Nicole                  NaN    Westerpark  52.37824    4.86826   
3  Danielle                  NaN          Zuid  52.34091    4.84802   
4    Marcel                  NaN  Centrum-West  52.37641    4.88303   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room     61               3                3

In order to give inside in what Amsterdam will receive from tourist tax if the event lasts a week and you will
have 30.000 visitors, we will calculate this using the mean 'price' for all the airbnb in Amsterdam. 
We extract tourist tax form the manucipality from Amsterdam: https://www.amsterdam.nl/veelgevraagd/toeristenbelasting-2c7c2#

In [15]:
# Calculate statistics
mean = df_air['price'].mean()
citytax_AMS = mean * 0.10
Income = citytax_AMS * 30000 * 7 
print('Income:', Income, 'City Tax:', citytax_AMS)

Income: 5344287.145242071 City Tax: 25.44898640591462


To give a clear overview we will plot the amount of AirBnB locations in Amsterdam per neighbourhood.

In [16]:
# Adding a column 
df_air['Number'] = range(len(df_air)) 
df_air['Number'] = 1
# Group by 'neighbourhood_group' and calculate the sum of 'price' for each group
neighbourhood = df_air.groupby('neighbourhood').sum().reset_index()
neighbourhood
# Create a bar plot using plotly
bar_fig = px.bar(neighbourhood, x='neighbourhood', y='Number', title="Airbnb by Neighbourhood in AMS")
bar_fig.update_xaxes(title_text="Neighbourhood")
bar_fig.update_yaxes(title_text="Number of Airbnb")
bar_fig.show()



Here we will find out which street has the most airbnb in Amsterdam.

In [ ]:
# Specify the city name (in this case, Amsterdam)
city = 'Amsterdam, Netherlands'

# Create a network graph for the specified city
G = (ox.graph_from_place(city))\
#Create a dictorairy
results = {}

for index, row in df_air.iterrows():
    lat = row['latitude']
    lon = row['longitude']
 
    # Find the nearest node to the specified coordinates
    nearest_edges= ox.distance.nearest_edges(G, lon, lat, return_dist=True)  

    if (nearest_edges[0][0], nearest_edges[0][1]) in results:
        results[(nearest_edges[0][0], nearest_edges[0][1])] += 1
    else:
        results[(nearest_edges[0][0], nearest_edges[0][1])] = 1

results


# Maximum value of the node
max(results, key=results.get)

# Convert the maxium node to the street name.

: 

We would like to make a cross reference between the BBGA data and the Airbnb data set. But first we will have to change the names of each neighbouhood in order to combine them. 

In [5]:
# Create a combined data frame with the Airbnb data and the BBGA data. 
df_BBGA = pd.read_excel('2023_BBGA_0614_cf8d825e89.xlsx')
df_air_BBGA = df_BBGA[['gebiednaam', 'sdnaam', 'BHVESTAIRBNB']]
# Change the names of the neighbourhood. 
BBGA_air = df_air_BBGA.iloc[3937:3962]
BBGA_air.to_excel("bbga_new.xlsx", index = False, header=1)
bbga_com = pd.read_excel('./bbga_new.xlsx')
bbga_com.replace({ "Oud West, De Baarsjes" :"De Baarsjes - Oud-West",
            "De Pijp, Rivierenbuurt":"De Pijp - Rivierenbuurt",
            "Indische Buurt, Oostelijk Havengebied" : "Oostelijk Havengebied - Indische Buurt",
            "IJburg, Zeeburgereiland" : "IJburg - Zeeburgereiland",
            "Geuzenveld, Slotermeer" : "Geuzenveld - Slotermeer",
            "Buitenveldert, Zuidas" : "Buitenveldert - Zuidas",
            "Oud-Zuid" : "Zuid" ,
            "De Aker, Sloten, Nieuw-Sloten" : "De Aker - Nieuw Sloten",
             "Gaasperdam" : "Gaasperdam - Driemond"}, inplace = True)

: 

: 

Next we can visualize the combined bar plot.

In [ ]:
# Visualize this in a bar plot.
fig_com = px.bar(bbga_com, x='gebiednaam', y='BHVESTAIRBNB', barmode='group', title= "Combined Airbnb by Neighbourhood in AMS")
fig_com.add_bar(x=neighbourhood['neighbourhood'], y=neighbourhood['Number'])
fig_com.update_xaxes(title_text="Neighbourhood")
fig_com.update_yaxes(title_text="Number of Airbnb")
fig_com.update_traces(name='Airbnb data', selector=dict(legendgroup='group1'))
fig_com.add_trace(go.Scatter(x=[None], y=[None], mode='markers', name='BBGA data of Airbnb'))
fig_com.update_layout(showlegend=True)
fig_com.show()

If we would like to figure out which appartments are not rented out all of the time but are used as normal housing we will set a limiet on 200 days. If the appartment is rented out more then 200 days a year it wont be used as housing. However, if the appartment is rented out less then 200 days, it will be used as housing. 

In [ ]:
# Filter the dataset for availability
df_above_200 = df_air[df_air['availability_365'] > 200]  #not used as housing 
df_below_200 = df_air[df_air['availability_365'] <= 200] #used as housing

# Calculate the sum for below or equal to 200 days, used as housing
sum_below_200 = df_below_200['Number'].sum()
print(sum_below_200)
# Calculate the sum for above 200 days, not used as housing
sum_above_200 = df_above_200['Number'].sum()
print(sum_above_200)

To asnwer the question about the different licenses we used the functions groupby and count when printed it shows the amount of different licenses.  

In [ ]:
# Amount of licenses
df_licenses = df_bnb.groupby('license').count()
print(len(df_licenses))
